In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyBigWig as bw
import os

In [45]:
data_folder='../results/binned_norm_counts/'
Samples = [f'PRO_SEQ_CT{4*i:02d}_S{i+1}_R1_001' for i in range(12)]
Strands = ['forward','reverse']
bin_size = 100
T = np.arange(0,48,4)

# awk '$3=="gene"' resources/genome/GRCm39/gencode.vM33.primary_assembly.annotation.gtf | grep "^chr" > resources/genome/GRCm39/gene.gtf
gtf = pd.read_csv('../resources/genome/GRCm39/gene.gtf',sep='\t',header=None)
gtf.columns = ['chr','source','type','start','end','score','strand','frame','attribute']
gtf['gene_name'] = gtf.attribute.str.extract(r'gene_name "(.*?)";')

f = {}
for sample in Samples:
    t = int(sample.split('_')[2][2:])
    f[t] = {}
    for strand in Strands:
        if strand=='forward':
            f[t]['+'] = bw.open(f"{data_folder}/{sample}/NormCoverage_3p_{strand}_bin{bin_size}bp.bw")
        elif strand=='reverse':
            f[t]['-'] = bw.open(f"{data_folder}/{sample}/NormCoverage_3p_{strand}_bin{bin_size}bp.bw")

for gene in ['Clock','Per2','Myc','Gapdh']:
    print(gene)

    coord = gtf.loc[gtf.gene_name==gene,['chr','start','end','strand']]
    chr = coord.chr.values[0]
    start = coord.start.values[0]
    end = coord.end.values[0]
    strand = coord.strand.values[0]

    Bins = np.arange(start - start%bin_size,end + bin_size - end%bin_size,bin_size)

    print(gene,chr,start,end,strand)

    pm_lfc = np.log2(np.mean( [i[2] for i in f[t]['+'].intervals(chr,start,end)] ) / np.mean( [i[2] for i in f[t]['-'].intervals(chr,start,end)] ))
    print(pm_lfc)

    for strand in ['+','-']:
        X = np.zeros((len(Bins),len(T)))
        X[:] = np.nan
        df = pd.DataFrame(X,index=Bins,columns=T)

        for t in T:
            vals = f[t][strand].intervals(chr,start,end)
            bins = [vals[i][0] for i in range(len(vals))]
            counts = [vals[i][2] for i in range(len(vals))]
            df.loc[bins,t] = counts

        idx_out = np.isnan(df.values).all(1)
        df = df.loc[~idx_out,:]
        df[np.isnan(df.values)] = 0

        X = df.values
        w = X.sum(1)
        w = w/w.sum()

        # fourier transform
        for n in range(1,2):
            f_n = np.sum(X*np.exp(-1j*2*n*np.pi*T/24),1)
            a_n = np.abs(f_n)
            phi_n = np.angle(f_n)

            w_f = np.mean(f_n*w)
            std_a = np.sqrt(np.sum(w*(a_n-w_a)**2))
            std_phi = np.sqrt(np.sum(w*(phi_n-w_phi)**2))
        
            print(n,np.round(w_a,3),np.round(std_a,3),np.round(w_phi,3),np.round(std_phi,3))



Clock
Clock chr5 76357715 76452639 -
0.9707049158608212
1 0.121 0.104 0.847 1.136
1 0.112 0.104 -0.528 1.734
Per2
Per2 chr1 91343704 91387046 -
-0.35431174078743094
1 0.517 0.493 1.246 2.571
1 0.212 0.176 -0.474 0.81
Myc
Myc chr15 61857240 61862223 +
-0.9259027485820506
1 0.033 0.013 -0.162 1.978
1 0.315 0.285 -1.302 0.501
Gapdh
Gapdh chr6 125138678 125143430 -
2.1965580884986555
1 1.112 0.73 -1.645 2.3
1 0.049 0.032 -0.552 1.804


In [64]:
gene = 'Per2'